In [66]:
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.saving import register_keras_serializable

In [67]:
california_housing = fetch_california_housing()
california_housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

# Splitting data into train and test.

In [68]:
X_train0, X_test, Y_train0, Y_test = train_test_split(california_housing["data"], california_housing["target"])

In [69]:
X_train1, X_validation, Y_train1, Y_validation = train_test_split(X_train0, Y_train0)

In [70]:
X_test.shape

(5160, 8)

In [71]:
X_train0.shape

(15480, 8)

In [72]:
X_train1.shape

(11610, 8)

In [73]:
X_validation.shape

(3870, 8)

# Normalizing data
using "sklearn.preprocessing.StandardScaler" function for normalizing data

In [74]:
sc = StandardScaler()
X_train_s = sc.fit_transform(X_train1)
X_validation_s = sc.fit_transform(X_validation)
X_test_s = sc.transform(X_test)

--------------------------------

In [75]:
X_train0.shape[1:]

(8,)

In [76]:
X_train_s_1, X_train_s_2 = X_train_s[:, :6], X_train_s[:, :-4]
X_validation_s_1, X_validation_s_2 = X_validation_s[:, :6], X_validation_s[:, :-4]
X_test_s_1, X_test_s_2 = X_test_s[:, :6], X_test_s[:, :-4]

# Subclass API

In [77]:
@register_keras_serializable()
class WideAndDeepANN(keras.Model):
    def __init__(self, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden_layer_1 = keras.layers.Dense(50, activation=activation)
        self.hidden_layer_2 = keras.layers.Dense(10, activation=activation)
        self.final_output = keras.layers.Dense(1)
        self.helper_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_1, input_2 = inputs
        h1_out = self.hidden_layer_1(input_1)
        h2_out = self.hidden_layer_2(h1_out)
        concat_out = keras.layers.concatenate([input_2, h2_out])
        final_out = self.final_output(concat_out)
        helper_out = self.helper_output(h2_out)
        return final_out, helper_out

In [78]:
model_f = WideAndDeepANN()

# Callback

In [79]:
model_checkpoint_callback = keras.callbacks.ModelCheckpoint("model_cb_regression_housing.keras", save_best_only=True)
early_stopping_callback = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [80]:
tb_callback = keras.callbacks.TensorBoard("tb_log")

# Compile Model

In [81]:
model_f.compile(loss=["mse", "mse"],
                loss_weights=[0.8, 0.2],
                optimizer="sgd",
                metrics=[["mean_absolute_error"], ["mean_absolute_error"]])

# Fit Model

In [82]:
model_f.fit((X_train_s_1, X_train_s_2), (Y_train1, Y_train1), epochs=5,
            validation_data=((X_validation_s_1, X_validation_s_2), (Y_validation, Y_validation)),
            callbacks=[tb_callback])

Epoch 1/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 1.8866 - mean_absolute_error: 0.9182 - mean_absolute_error_1: 1.1471 - mse_loss: 2.4511 - val_loss: 0.5687 - val_mean_absolute_error: 0.5427 - val_mean_absolute_error_1: 0.5958 - val_mse_loss: 0.6140
Epoch 2/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.5956 - mean_absolute_error: 0.5585 - mean_absolute_error_1: 0.6008 - mse_loss: 0.6442 - val_loss: 0.5632 - val_mean_absolute_error: 0.5591 - val_mean_absolute_error_1: 0.5713 - val_mse_loss: 0.5596
Epoch 3/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.5725 - mean_absolute_error: 0.5506 - mean_absolute_error_1: 0.5611 - mse_loss: 0.5836 - val_loss: 0.5471 - val_mean_absolute_error: 0.5527 - val_mean_absolute_error_1: 0.5747 - val_mse_loss: 0.5587
Epoch 4/5
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.5576 - mean_absolute_error: 0.5402 - mean_absolute_error_1: 0.5433 - mse_loss: 0.5634 - val_loss: 0.6076 - val_mean_absolute_error: 0.5923 - val_mean_absolute_err

In [83]:
model_f.save("housing_reg_model.keras")

In [84]:
model_f_reg = keras.models.load_model("housing_reg_model.keras")

In [85]:
model_f_reg.summary()

Model: "wide_and_deep_ann_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_28 (Dense)                │ (None, 50)             │           350 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 10)             │           510 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 1)              │            15 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 888 (3.47 KB)

 Trainable params: 886 (3.46 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [86]:
class Mycallback(keras.callbacks.Callback):
    def on_train_end(self, logs=None):
        print("")
    def on_epoch_end(self, epoch, logs):
        print(logs["var_loss"])

In [87]:
mycb = Mycallback